## **Schemna Inicial**

In [ ]:
# 2. Carga librerías y tu JSON
import json
import pandas as pd
import pandera as pa
from pandera import Column, Check, DataFrameSchema

with open('repos.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

date_cols = ["created_at", "updated_at", "pushed_at"]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], format="%Y-%m-%dT%H:%M:%SZ", errors="raise")

print(df[date_cols].dtypes)

# 3. Define el esquema de validación con Pandera
schema = DataFrameSchema({
    "id":                 Column(int,   Check.ge(0)),
    "node_id":            Column(str,   nullable=False),
    "name":               Column(str,   Check.str_length(min_value=1)),
    "full_name":          Column(str,   Check.str_length(min_value=1)),
    "private":            Column(bool),
    "html_url":           Column(str,   Check.str_matches(r"^https?://github\.com/")),
    "description":        Column(str,   nullable=True),
    "fork":               Column(bool),
    "url":                Column(str,   Check.str_matches(r"^https?://api\.github\.com/")),
    "created_at":         Column(pa.DateTime),
    "updated_at":         Column(pa.DateTime),
    "pushed_at":          Column(pa.DateTime),
    "size":               Column(int,   Check.ge(0)),
    "stargazers_count":   Column(int,   Check.ge(0)),
    "watchers_count":     Column(int,   Check.ge(0)),
    "language":           Column(str,   nullable=True),
    "has_issues":         Column(bool),
    "forks_count":        Column(int,   Check.ge(0)),
    "open_issues_count":  Column(int,   Check.ge(0)),
    "default_branch":     Column(str,   Check.str_length(min_value=1))
})

# 4. Ejecuta la validación (lazy=True para reportar todos los errores)
schema.validate(df, lazy=True)

## **Conversión númerica y Visualización de Nan**

In [ ]:
# ——————————————————————————
# Paso 2 (completo): Conversión de **todas** las columnas numéricas y detección de errores
# ——————————————————————————

import pandas as pd

# 1. Detecta automáticamente todas las columnas numéricas (int y float)
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
print("Columnas numéricas a convertir:", numeric_cols)

# 2. Convertir a entero
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce", downcast="integer")

# 3. Reporte: ¿cuántos NaN quedaron tras la conversión en cada columna?
nan_report = df[numeric_cols].isna().sum()
print("\nValores no convertidos a numérico (NaN) por columna:\n")
print(nan_report)

# 4. Para cada columna con NaN,
for col, n in nan_report.items():
    if n > 0:
        print(f"\n>>> Columna '{col}' tiene {n} valores NaN tras conversión. Ejemplos:")
        print(df[df[col].isna()][["id", col]].head(), "\n")


## **Completitud**

In [ ]:
# ——————————————————————————
# Paso 3: Completitud
# ——————————————————————————

import pandas as pd

# 1. Conteo y porcentaje de valores nulos por columna
missing_count = df.isna().sum()
missing_pct   = df.isna().mean()
completitud = pd.concat([missing_count, missing_pct], axis=1)
completitud.columns = ["n_missing", "pct_missing"]
print("Resumen de valores faltantes:\n")
print(completitud.sort_values("pct_missing", ascending=False))

# 2. Definir un umbral de tolerancia (p. ej. 5 %)
umbral = 0.05
cols_altos = completitud[completitud["pct_missing"] > umbral].index.tolist()
print(f"\nColumnas con > {umbral*100:.0f}% faltantes:\n", cols_altos)


# para 'description' usar cadena vacía:
if "description" in df.columns:
    df["description"] = df["description"].fillna("")

# Para booleans, se imputa False:
bool_cols = df.select_dtypes(include="bool").columns
for col in bool_cols:
    df[col] = df[col].fillna(False)

# Para fechas, se imputa fecha mínima o actual:
date_cols = ["created_at", "updated_at", "pushed_at"]
for col in date_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].min())

# 4. Se verifica de nuevo que no queden nulos en columnas críticas
print("\nChequeo tras imputación parcial:")
print(df[["description"] + bool_cols.tolist() + date_cols].isna().sum())


## **Paso 3b manejar la completitud**

In [ ]:
# ——————————————————————————
# Paso 3b: Tratar columnas con muchos nulos
# ——————————————————————————

# 1. Eliminar columnas totalmente vacías
df = df.drop(columns=["mirror_url", "license"])

# 2. Imputar homepage y language
df["homepage"] = df["homepage"].fillna("")           # campo de URL vacío
df["language"] = df["language"].fillna("Unknown")    # placeholder para lenguajes faltantes

# 3. Verificar de nuevo
cols_check = ["description", "homepage", "language"]
print("\nValores faltantes tras imputación/dropping:")
print(df[cols_check].isna().sum())


In [ ]:
# ——————————————————————————
# Paso 4: Detección y eliminación de duplicados ignorando columnas anidadas
# ——————————————————————————

import pandas as pd

# 1. Identificar columnas con valores dict o list
nested_cols = [
    col
    for col in df.columns
    if df[col].apply(lambda x: isinstance(x, (dict, list))).any()
]
print("Columnas anidadas (se excluirán de la comparación):", nested_cols)

# 2. Crear un DataFrame temporal sin esas columnas
df_temp = df.drop(columns=nested_cols)

# 3. Contar filas duplicadas en df_temp
dup_rows = df_temp.duplicated()
print(f"Número de filas duplicadas (sin columnas anidadas): {dup_rows.sum()}")

# 4. Si hay duplicados completos, ver ejemplos
if dup_rows.sum() > 0:
    print("\nEjemplos de filas duplicadas en el original:")
    display(df[dup_rows].head())

# 5. Eliminar duplicados completos del DataFrame original
df = df[~dup_rows].reset_index(drop=True)

# 6. Verificar duplicados en la clave primaria 'id'
id_dup = df["id"].duplicated()
print(f"\nNúmero de IDs repetidos: {id_dup.sum()}")

# 7. Mostrar IDs repetidos si los hay
if id_dup.sum() > 0:
    print("\nIDs duplicados y sus registros:")
    display(df[id_dup][["id", "name", "full_name"]])

# 8. Eliminar filas con IDs duplicados (manteniendo la primera aparición)
df = df[~id_dup].reset_index(drop=True)

# 9. Confirmar que ya no quedan duplicados
print(f"\nDuplicados completos tras limpieza: {df_temp.duplicated().sum()}")
print(f"IDs repetidos tras limpieza: {df['id'].duplicated().sum()}")


In [ ]:
# ——————————————————————————
# Paso 5: Consistencia entre columnas
# ——————————————————————————

import pandas as pd

# 1. watchers_count vs watchers
mask_watchers = df["watchers_count"] != df["watchers"]
print(f"watchers_count != watchers: {mask_watchers.sum()} registros")
if mask_watchers.any():
    display(df.loc[mask_watchers, ["id", "watchers_count", "watchers"]])

# 2. forks_count vs forks
mask_forks = df["forks_count"] != df["forks"]
print(f"forks_count != forks: {mask_forks.sum()} registros")
if mask_forks.any():
    display(df.loc[mask_forks, ["id", "forks_count", "forks"]])

# 3. open_issues_count vs open_issues
mask_issues = df["open_issues_count"] != df["open_issues"]
print(f"open_issues_count != open_issues: {mask_issues.sum()} registros")
if mask_issues.any():
    display(df.loc[mask_issues, ["id", "open_issues_count", "open_issues"]])

# 4. Fecha relaciones
mask_pushed_before_created = df["pushed_at"] < df["created_at"]
print(f"pushed_at < created_at: {mask_pushed_before_created.sum()} registros")
if mask_pushed_before_created.any():
    display(df.loc[mask_pushed_before_created, ["id", "created_at", "pushed_at", "updated_at"]])

mask_updated_before_created = df["updated_at"] < df["created_at"]
print(f"updated_at < created_at: {mask_updated_before_created.sum()} registros")
if mask_updated_before_created.any():
    display(df.loc[mask_updated_before_created, ["id", "created_at", "updated_at", "pushed_at"]])

# 5. Validación de homepage
mask_homepage_invalid = (df["homepage"] != "") & (~df["homepage"].str.match(r"^https?://"))
print(f"homepage no vacío pero inválido: {mask_homepage_invalid.sum()} registros")
if mask_homepage_invalid.any():
    display(df.loc[mask_homepage_invalid, ["id", "homepage"]])


## **Corregir Consistencia**

In [ ]:
# ——————————————————————————
# Paso 5b: Corregir anomalies de pushed_at < created_at
# ——————————————————————————

# 1. Identificar el mask de inconsistencia
mask = df["pushed_at"] < df["created_at"]
print(f"Registros a corregir (pushed_at < created_at): {mask.sum()}")

# 2. Corregir: igualar pushed_at a created_at
df.loc[mask, "pushed_at"] = df.loc[mask, "created_at"]

# 3. Verificar que ya no haya inconsistencias
mask2 = df["pushed_at"] < df["created_at"]
print(f"Registros inconsistentes tras corrección: {mask2.sum()}")


## **Puntualidad**

In [ ]:
# ——————————————————————————
# Paso 6: Puntualidad
# ——————————————————————————

import pandas as pd


now = pd.Timestamp.now()
df["age_days"] = (now - df["updated_at"]).dt.days


print("Estadísticas de edad (días) de los repositorios:\n")
print(df["age_days"].describe())


threshold = 3650
stale_10y = df[df["age_days"] > threshold]
print(f"\nRepositorios sin actualizar en más de 10 años (> {threshold} días): {stale_10y.shape[0]}")

# 4. Mostrar algunos ejemplos
if not stale_10y.empty:
    display(stale_10y[["id", "full_name", "updated_at", "age_days"]].head())




## **Corregir Puntualidad**

In [ ]:
# ——————————————————————————
# Paso 6b: Eliminar repositorios sin actualizar en más de 10 años
# ——————————————————————————


threshold = 3650  # días

# Filtra el DataFrame solo con los repos "puntuales"
df = df[df["age_days"] <= threshold].reset_index(drop=True)


stale_after = df[df["age_days"] > threshold]
print(f"Repositorios obsoletos tras limpieza: {stale_after.shape[0]}")


df = df.drop(columns=["age_days"])


## **Normalización**

In [ ]:
# ——————————————————————————
# Paso 7 : Normalización de cadenas, excluyendo columnas anidadas
# ——————————————————————————

import re

# 1. Detectar columnas anidadas (dict o list)
nested_cols = [
    col
    for col in df.columns
    if df[col].apply(lambda x: isinstance(x, (dict, list))).any()
]
print("Se excluyen de normalización:", nested_cols)

# 2. Seleccionar sólo las columnas object que NO sean anidadas
str_cols = [
    col for col in df.select_dtypes(include=['object']).columns
    if col not in nested_cols
]
print("Columnas a normalizar:", str_cols)

# 3. Para cada columna de texto:
for col in str_cols:
    # a) Quitar espacios al inicio/fin y reducir espacios múltiples
    df[col] = df[col].apply(
        lambda x: re.sub(r'\s+', ' ', x.strip()) if isinstance(x, str) else x
    )

# 4. Unificar a minúsculas campos categóricos
for col in ['name', 'full_name', 'default_branch', 'language']:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: x.lower() if isinstance(x, str) else x)

# 5. Mostrar un vistazo tras la normalización
print("Ejemplo de normalización:")
display(df.loc[:, str_cols + ['name', 'full_name', 'language']].head())


## **Desanidar Owner**

In [ ]:
# ——————————————————————————
# Paso 8: “Desanidar” la columna `owner`
# ——————————————————————————

# 1. Verifica que la columna owner siga presente
if "owner" not in df.columns:
    raise KeyError("No encuentro la columna 'owner' en el DataFrame.")

# 2. Usa json_normalize para expandir el dict en un DataFrame aparte
owner_df = pd.json_normalize(df["owner"].dropna()).add_prefix("owner_")

# 3. Asegura que los índices coincidan
owner_df.index = df.index

# 4. Elimina la columna original y concatena las nuevas columnas
df = pd.concat([df.drop(columns=["owner"]), owner_df], axis=1)

# 5. Comprueba el resultado
cols_owner = [c for c in df.columns if c.startswith("owner_")]
print("Columnas desanidadas de owner:", cols_owner)
display(df.head())

## **Exportar limpio**

In [ ]:
# ——————————————————————————
# Paso 9: Exportar DataFrame limpio
# ——————————————————————————

# 1. Exportar a CSV
df.to_csv('repos_clean.csv', index=False, encoding='utf-8')
print("→ Archivo exportado: repos_clean.csv")

# 2.  Exportar a Excel
df.to_excel('repos_clean.xlsx', index=False)
print("→ Archivo exportado: repos_clean.xlsx")